Starting Step 2

Phone Verify Account to Turn on Internet (Session options)

Once GPU enabled, use T4 or P100.

first thing before you run the code, "pip install ultralytics" in the console at the bottom

make sure you turn on draft session (with accelerator) before you start training and don't turn it off until you have all the data.

In [4]:
###############################################################
# EXTRACT ULTRALYTICS WHEEL & IMPORT LOCALLY (KAGGLE SAFE
###############################################################

import os
import zipfile
import glob
import sys
import shutil

print("Searching for ultralytics wheel...")

# Locate the uploaded wheel file
wheel_files = glob.glob("/kaggle/input/**/*.whl", recursive=True)

if not wheel_files:
    raise FileNotFoundError("No .whl file found. Please upload an ultralytics-8.3.x wheel file.")

WHEEL_PATH = wheel_files[0]
print("Wheel located at:", WHEEL_PATH)

# Define extraction directory
EXTRACT_DIR = "/kaggle/working/ultralytics_lib"

# Remove previous extraction directory if it exists
if os.path.exists(EXTRACT_DIR):
    print("Removing previous extraction directory...")
    shutil.rmtree(EXTRACT_DIR)

# Extract the wheel contents
print("Extracting wheel contents...")
with zipfile.ZipFile(WHEEL_PATH, "r") as z:
    z.extractall(EXTRACT_DIR)

# Add extracted directory to Python path
sys.path.insert(0, EXTRACT_DIR)

print("Importing Ultralytics from extracted wheel...")
from ultralytics import YOLO

print("Ultralytics successfully imported without pip installation.")


Searching for ultralytics wheel...
Wheel located at: /kaggle/input/ultralytics-8-3-232/ultralytics-8.3.232-py3-none-any.whl
Removing previous extraction directory...
Extracting wheel contents...
Importing Ultralytics from extracted wheel...
Ultralytics successfully imported without pip installation.


In [ ]:
###############################################
# AER850 – PROJECT 3 – KAGGLE FULL PIPELINE
# YOLOv11 + PCB Masking + Training + Evaluation
# OOM-SAFE CONFIG FOR TESLA T4
###############################################

import os
import glob
import shutil
import random
import zipfile
import sys
import cv2
import torch
import numpy as np

# ------------------------------------------------------------
# 0. Install / import Ultralytics from wheel (offline)
# ------------------------------------------------------------

def install_ultralytics_from_wheel():
    wheel_files = glob.glob("/kaggle/input/**/*.whl", recursive=True)
    if not wheel_files:
        raise FileNotFoundError("Ultralytics .whl not found in /kaggle/input.")
    wheel_path = wheel_files[0]

    extract_dir = "/kaggle/working/ultralytics_lib"
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir, exist_ok=True)
        with zipfile.ZipFile(wheel_path, "r") as z:
            z.extractall(extract_dir)

    if extract_dir not in sys.path:
        sys.path.insert(0, extract_dir)

    from ultralytics import YOLO
    return YOLO


YOLO = install_ultralytics_from_wheel()

# ------------------------------------------------------------
# 1. Global paths / settings
# ------------------------------------------------------------

INPUT_ROOT = "/kaggle/input"
PROJECT_ROOT = "/kaggle/working"
MASK_OUTPUT_DIR = os.path.join(PROJECT_ROOT, "mask_outputs")
FINAL_OUTPUT_DIR = os.path.join(PROJECT_ROOT, "project3_outputs")
RUNS_DETECT_DIR = os.path.join(PROJECT_ROOT, "runs", "detect")

MODEL_NAME = "pcb_model_best_kaggle"

IMG_SIZE = 1280
BATCH_SIZE = 4
EPOCHS = 200
EARLY_STOP_PATIENCE = 20

RUN_MASKING = True
RUN_TRAINING = True
RUN_EVALUATION = True

SEED = 42

# ------------------------------------------------------------
# 2. Seed / device setup
# ------------------------------------------------------------

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    try:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    except:
        pass

def get_device():
    if torch.cuda.is_available():
        return "cuda"
    return "cpu"

# ------------------------------------------------------------
# 3. Finders: data.yaml, motherboard image, weights, eval images
# ------------------------------------------------------------

def find_data_yaml():
    paths = glob.glob(os.path.join(INPUT_ROOT, "**", "data.yaml"), recursive=True)
    if not paths:
        raise FileNotFoundError("data.yaml not found in /kaggle/input.")
    return paths[0]

def find_motherboard_image():
    images = []
    for ext in ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG"):
        images.extend(glob.glob(os.path.join(INPUT_ROOT, "**", ext), recursive=True))
    matches = [f for f in images if "motherboard" in os.path.basename(f).lower()]
    if not matches:
        raise FileNotFoundError("No motherboard image found in /kaggle/input.")
    return matches[0]

def find_yolo_weights():
    paths = glob.glob(os.path.join(INPUT_ROOT, "**", "yolo*.pt"), recursive=True)
    if not paths:
        raise FileNotFoundError("YOLO weights not found in /kaggle/input.")
    return paths[0]

def find_evaluation_images():
    images = []
    for ext in ("*.jpg","*.jpeg","*.png","*.JPG","*.JPEG","*.PNG"):
        for f in glob.glob(os.path.join(INPUT_ROOT, "**", ext), recursive=True):
            if os.path.basename(os.path.dirname(f)).lower() == "evaluation":
                images.append(f)
    if not images:
        raise FileNotFoundError("No evaluation images found in folder named 'evaluation'.")
    return sorted(images)

# ------------------------------------------------------------
# 4. Step 1 — PCB masking
# ------------------------------------------------------------

def step1_masking(motherboard_path):
    os.makedirs(MASK_OUTPUT_DIR, exist_ok=True)

    img = cv2.imread(motherboard_path)
    if img is None:
        raise FileNotFoundError("Could not read motherboard image.")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    edges = cv2.Canny(blur, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        raise RuntimeError("No contours detected.")

    largest = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [largest], -1, 255, -1)
    pcb = cv2.bitwise_and(img, img, mask=mask)

    cv2.imwrite(os.path.join(MASK_OUTPUT_DIR, "01_original.jpg"), img)
    cv2.imwrite(os.path.join(MASK_OUTPUT_DIR, "02_gray.jpg"), gray)
    cv2.imwrite(os.path.join(MASK_OUTPUT_DIR, "03_blur.jpg"), blur)
    cv2.imwrite(os.path.join(MASK_OUTPUT_DIR, "04_edges.jpg"), edges)
    cv2.imwrite(os.path.join(MASK_OUTPUT_DIR, "05_mask.jpg"), mask)
    cv2.imwrite(os.path.join(MASK_OUTPUT_DIR, "06_pcb_extracted.jpg"), pcb)

# ------------------------------------------------------------
# 5. Step 2 — Training YOLO (OOM-safe)
# ------------------------------------------------------------

def step2_training(device, yaml_path):
    weights_path = find_yolo_weights()
    model = YOLO(weights_path)

    os.makedirs(RUNS_DETECT_DIR, exist_ok=True)

    model.train(
        data=yaml_path,
        imgsz=IMG_SIZE,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        device=device,
        workers=1,
        amp=False,
        mosaic=0.0,
        fliplr=0.3,
        flipud=0.2,
        hsv_h=0.015,
        patience=EARLY_STOP_PATIENCE,
        cache=False,
        plots=True,
        project=RUNS_DETECT_DIR,
        name=MODEL_NAME,
        exist_ok=True,
    )

    run_folder = os.path.join(RUNS_DETECT_DIR, MODEL_NAME)

    metrics = model.val(
        data=yaml_path,
        imgsz=IMG_SIZE,
        device=device,
        save=True,
        plots=True,
        project=RUNS_DETECT_DIR,
        name=MODEL_NAME,
        exist_ok=True,
    )

    with open(os.path.join(run_folder, "metrics_summary.txt"), "w") as f:
        f.write(str(metrics))

    return run_folder

# ------------------------------------------------------------
# 6. Step 3 — Evaluation
# ------------------------------------------------------------

def step3_eval(device, run_folder):
    best_weights = os.path.join(run_folder, "weights", "best.pt")
    if not os.path.isfile(best_weights):
        raise FileNotFoundError("best.pt not found; training may not have finished.")

    model = YOLO(best_weights)
    eval_images = find_evaluation_images()

    model.predict(
        source=eval_images,
        device=device,
        save=True,
        save_txt=True,
        project=RUNS_DETECT_DIR,
        name="pcb_eval",
        exist_ok=True,
        imgsz=IMG_SIZE,
        conf=0.25,
    )

# ------------------------------------------------------------
# 7. Step 4 — Output consolidation
# ------------------------------------------------------------

def consolidate_outputs(run_folder):
    if os.path.exists(FINAL_OUTPUT_DIR):
        shutil.rmtree(FINAL_OUTPUT_DIR)
    os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)

    if os.path.exists(MASK_OUTPUT_DIR):
        shutil.copytree(MASK_OUTPUT_DIR, os.path.join(FINAL_OUTPUT_DIR, "masking"))

    if os.path.exists(run_folder):
        shutil.copytree(run_folder, os.path.join(FINAL_OUTPUT_DIR, "training_outputs"))

    eval_out = os.path.join(RUNS_DETECT_DIR, "pcb_eval")
    if os.path.exists(eval_out):
        shutil.copytree(eval_out, os.path.join(FINAL_OUTPUT_DIR, "evaluation_outputs"))

    with open(os.path.join(FINAL_OUTPUT_DIR, "report_notes.txt"), "w") as f:
        f.write(
            "AER850 – Project 3 Output Structure\n"
            "masking/              PCB masking steps\n"
            "training_outputs/     YOLO training results\n"
            "evaluation_outputs/   Evaluation detections\n"
        )

# ------------------------------------------------------------
# 8. Main
# ------------------------------------------------------------

if __name__ == "__main__":
    set_seed(SEED)
    device = get_device()

    yaml_path = find_data_yaml()
    motherboard_img = find_motherboard_image()

    if RUN_MASKING:
        step1_masking(motherboard_img)

    if RUN_TRAINING:
        run_folder = step2_training(device, yaml_path)
    else:
        run_folder = os.path.join(RUNS_DETECT_DIR, MODEL_NAME)

    if RUN_EVALUATION:
        step3_eval(device, run_folder)

    consolidate_outputs(run_folder)

    print("Project 3 completed.")


New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.232 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/aer850-project-3-data-new/Project 3 Data - Kaggle/Project 3 Data/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.3, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/yolo-weight

In [ ]:
"""
AER850 – Project 3
KAGGLE FINAL DETECTION SCRIPT
---------------------------------------------------------------
"""

# ---------------------------------------------------------------
# 0. LOAD ULTRALYTICS FROM WHEEL
# ---------------------------------------------------------------

import os, glob, zipfile, sys, cv2, yaml, torch
import numpy as np
import matplotlib.pyplot as plt

print("Searching for Ultralytics wheel...")

wheel_files = glob.glob("/kaggle/input/**/*.whl", recursive=True)
if not wheel_files:
    raise FileNotFoundError("Ultralytics .whl file not found in /kaggle/input.")

WHEEL_PATH = wheel_files[0]
print("Found wheel:", WHEEL_PATH)

EXTRACT_DIR = "/kaggle/working/ultralytics_lib"
if os.path.exists(EXTRACT_DIR):
    import shutil
    shutil.rmtree(EXTRACT_DIR)

print("Extracting wheel...")
with zipfile.ZipFile(WHEEL_PATH, "r") as z:
    z.extractall(EXTRACT_DIR)

sys.path.insert(0, EXTRACT_DIR)

from ultralytics import YOLO
print("Ultralytics imported successfully.")


# ---------------------------------------------------------------
# 1. GLOBAL PATHS
# ---------------------------------------------------------------

INPUT_ROOT = "/kaggle/input"
WORK_ROOT = "/kaggle/working"
SAVE_DIR = os.path.join(WORK_ROOT, "prediction_outputs")

os.makedirs(SAVE_DIR, exist_ok=True)
print("Saving predictions to:", SAVE_DIR)


# ---------------------------------------------------------------
# 2. FINDERS
# ---------------------------------------------------------------

def find_best_pt():
    """Locate best.pt under expected training path, /kaggle/working, or /kaggle/input."""
    print("\nSearching for best.pt...")

    direct = "/kaggle/working/runs/detect/pcb_model_best_kaggle/weights/best.pt"
    if os.path.exists(direct):
        print("Found best.pt at:", direct)
        return direct

    working_list = glob.glob("/kaggle/working/**/best.pt", recursive=True)
    if working_list:
        print("Found best.pt under /kaggle/working:", working_list[0])
        return working_list[0]

    input_list = glob.glob("/kaggle/input/**/best.pt", recursive=True)
    if input_list:
        print("Found best.pt under /kaggle/input:", input_list[0])
        return input_list[0]

    raise FileNotFoundError(
        "best.pt not found. Training may not have completed or run folder not attached."
    )


def find_data_yaml():
    """Locate data.yaml anywhere under /kaggle/input."""
    print("\nSearching for data.yaml...")
    files = glob.glob("/kaggle/input/**/data.yaml", recursive=True)
    if not files:
        raise FileNotFoundError("data.yaml not found under /kaggle/input.")
    print("Using data.yaml:", files[0])
    return files[0]


def find_evaluation_images():
    """Locate images inside folders named 'evaluation' under /kaggle/input."""
    print("\nSearching for evaluation images...")
    result = []
    exts = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]

    for ext in exts:
        pattern = os.path.join("/kaggle/input", "**", ext)
        for item in glob.glob(pattern, recursive=True):
            parent = os.path.basename(os.path.dirname(item)).lower()
            if parent == "evaluation":
                result.append(item)

    if not result:
        raise RuntimeError("No evaluation images found in a folder named 'evaluation'.")

    print("Found evaluation images:", len(result))
    return sorted(result)


def load_class_names(path):
    """Load CLASS_NAMES from data.yaml."""
    with open(path, "r") as f:
        data = yaml.safe_load(f)
    names = data.get("names")
    if names is None:
        raise RuntimeError("Missing 'names' in data.yaml.")
    print("Class names loaded:", names)
    return names


# ---------------------------------------------------------------
# 3. SMALL CLEAN LABEL DRAWING
# ---------------------------------------------------------------

COLOR_LIST = [
    (0, 255, 0),
    (255, 128, 0),
    (255, 0, 0),
    (255, 255, 0),
    (255, 0, 255),
    (128, 0, 255),
]

def draw_clean_boxes(img, results, names):
    """
    Draw compact bounding boxes and adaptive small labels.
    Ensures minimal obstruction of PCB features.
    """
    h, w = img.shape[:2]

    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        cls = int(box.cls)
        conf = float(box.conf)
        text = f"{names[cls]} {conf:.2f}"

        color = COLOR_LIST[cls % len(COLOR_LIST)]
        scale = max(0.3, min(w / 2800, h / 2800))
        thickness = max(1, int(scale * 2))

        (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        cv2.rectangle(img, (x1, y1 - th - 6), (x1 + tw + 6, y1), color, -1)
        cv2.putText(
            img, text, (x1 + 3, y1 - 4),
            cv2.FONT_HERSHEY_SIMPLEX, scale, (0, 0, 0),
            max(1, thickness - 1),
        )

    return img


# ---------------------------------------------------------------
# 4. LOAD MODEL AND FILES
# ---------------------------------------------------------------

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("\nUsing device:", DEVICE)

BEST_PT = find_best_pt()
DATA_YAML = find_data_yaml()
EVAL_IMGS = find_evaluation_images()
CLASS_NAMES = load_class_names(DATA_YAML)

print("\nLoading trained YOLO model...")
model = YOLO(BEST_PT)
print("Model loaded.")


# ---------------------------------------------------------------
# 5. RUN DETECTION
# ---------------------------------------------------------------

print("\nRunning detection...\n")

total_count = 0
per_class = {}

for path in EVAL_IMGS:
    name = os.path.basename(path)
    print("Processing:", name)

    img = cv2.imread(path)
    if img is None:
        print("Could not read image:", path)
        continue

    results = model.predict(img, device=DEVICE, conf=0.25, verbose=False)[0]

    for box in results.boxes:
        cls_name = CLASS_NAMES[int(box.cls)]
        per_class[cls_name] = per_class.get(cls_name, 0) + 1
        total_count += 1

    img = draw_clean_boxes(img, results, CLASS_NAMES)

    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(rgb)
    plt.axis("off")
    plt.title(name)
    plt.show()

    out_path = os.path.join(SAVE_DIR, "det_" + name)
    cv2.imwrite(out_path, img)
    print("Saved:", out_path)


# ---------------------------------------------------------------
# 6. SUMMARY
# ---------------------------------------------------------------

print("\nDetection Summary")
print("------------------")
print("Total detections:", total_count)
print()

for cls, count in per_class.items():
    print(f"{cls:20s}: {count}")

print("\nAll predictions saved in:", SAVE_DIR)
